# Table to make schema figure of Maatbase database

**Motivation:** Analyze whole database. Find all primary and foreying keys. Find connection between tables. This is analysis for later database schema creation

### imports and loads

In [1]:
import numpy as np
import pandas as pd
import set_path
import supp.support_load as sl

In [2]:
dfs, dfs_name, dfs_export_date = sl.load_pickle()
iton, ntoi = sl.get_name_dicts(dfs_name)
dfs_export_date

Pickle database loaded.


'2024-02-19'

### for each table find all keys and other columns

In [3]:
import pandas as pd
import pickle
import sqlalchemy as sa
from sqlalchemy import MetaData, Table
from sqlalchemy import inspect

# Load data from the pickle file (assuming it's a list of DataFrames)
data = dfs

# Step 2: Create an in-memory SQLite database
engine = sa.create_engine('sqlite:///:memory:')
metadata = MetaData()

# Step 3: Convert each DataFrame in the list to a SQLAlchemy table
for i, df in enumerate(data):
    table_name = f'{str(i).zfill(2)}_{iton[i]}'
    df.to_sql(table_name, engine, if_exists='replace', index=False)

# Step 4: Inspect the schema
inspector = inspect(engine)

table_dict = {}
for table_name in inspector.get_table_names():
    #print(f"\nTable: {table_name}")
    table_columns = {}
    for column in inspector.get_columns(table_name):
        #print(f"Column: {column['name']}, Type: {column['type']}")
        table_columns[str(column['name'])] = str(column['type'])
    table_dict[table_name] = table_columns
print(len(table_dict))

44


In [4]:
import re

def contains_id(text):
    return bool(re.search(r'(^|_)ID(_|$)', text, re.IGNORECASE))

In [5]:
table_id_dict = {}
for table_name, value in table_dict.items():
    #print(f"\nTable: {table_name}")
    id_list = []
    for column, type in value.items():
        if contains_id(column):
            #print(f'\t\t{column}')
            id_list.append(column)
    table_id_dict[table_name] = sorted(id_list)

exception = {
    '23_df_kings_info': ['king'],
    '00_df_general': ['king'], 
}
for key, value in exception.items():
    table_id_dict[key] = sorted(table_id_dict[key] + exception[key])

print(len(table_id_dict))

44


In [6]:
table_id_dict

{'00_df_general': ['ID_official',
  'ID_person',
  'king',
  'name_id',
  'tomb_code_id'],
 '01_df_tomb': ['tomb_code_id'],
 '02_df_tomb_king_name': ['ID_official', 'tomb_code_id'],
 '03_df_tomb_chapel': ['tomb_code_id'],
 '04_df_tomb_osiris': ['tomb_code_id'],
 '05_df_tomb_statue_material': ['tomb_code_id'],
 '06_df_tomb_dim': ['tomb_code_id'],
 '07_df_name': ['ID_official_source', 'name_id'],
 '08_df_person': ['ID_person'],
 '09_df_person_working_activity': ['ID_person'],
 '10_df_shaft_burial_chamber': ['ID_official',
  'coffin_ID',
  'sarcophagus_ID',
  'shaft_ID'],
 '11_df_shaft_sarcophagus': ['sarcophagus_ID'],
 '12_df_shaft_coffin': ['coffin_ID'],
 '13_df_tomb_shaft_relation': ['ID_person', 'shaft_ID', 'tomb_code_id'],
 '14_df_shaft_burial_chamber_dim': ['shaft_ID'],
 '15_df_body': ['shaft_ID'],
 '16_df_body_housed': ['shaft_ID'],
 '17_df_source_type': ['ID_official'],
 '18_df_notes': ['ID_official'],
 '19_df_current_location': ['ID_official'],
 '20_df_dating': ['ID_official'],
 

### validate keys with excel file

In [7]:
path = r'C:\Users\Stoja\OneDrive\Documents\diplomka\scr\excel_dfs_description\frozen_panels\maatbase_digest_overview.xlsx'
df_excel = pd.read_excel(path)
df_excel['primary_key_list'] = df_excel['primary key'].map(lambda x: x.split(', ') if isinstance(x, str) else [])
df_excel['foreing_key_list'] = df_excel['foreing keys'].map(lambda x: x.split(', ') if isinstance(x, str) else [])
df_excel['all_key_list'] = df_excel.apply(lambda x: x['primary_key_list'] + x['foreing_key_list'], axis=1)
df_excel['all_key_list'] = df_excel['all_key_list'].map(lambda x: sorted(x))

In [8]:
df_excel[['number', 'table', 'all_key_list']]
table_id_excel = {}
primary_keys = {}
for id, row in df_excel.iterrows():
    table_name = f"{str(row['number']).zfill(2)}_{row['table']}"
    table_id_excel[table_name] = row['all_key_list']
    primary_keys[table_name] = row['primary_key_list']
print(len(table_id_excel))

44


In [9]:
list(table_id_excel.keys()) == list(table_id_dict.keys())

True

In [10]:
keys_diff = []
for key in table_id_excel.keys():
    list_excel = table_id_excel[key]
    list_dict = table_id_dict[key]
    if list_excel!=list_dict:
        print(f'{list_excel==list_dict}\t{key}')
        keys_diff.append(key)

print('\n', keys_diff)
for key in keys_diff:
    print(table_id_excel[key])
    print(table_id_dict[key])

False	34_df_family_relations

 ['34_df_family_relations']
['ID_family_relation', 'ID_official (2x)', 'ID_person (2x)']
['ID_family_relation', 'ID_official_1', 'ID_official_2', 'ID_person_1', 'ID_person_2']


### make body script for dbdiagram.io

In [11]:
columns_all = table_dict.copy()
primary_key = primary_keys.copy()
foreing_key = table_id_dict.copy()

In [12]:
table_code_example = """Table TABLE_NAME {
TABLE_BODY
}"""
primary_key_example = '\tPRIMARY_KEY COLUMN_TYPE [primary key]'
colum_example = '\tCOLUMN_NAME COLUMN_TYPE'

In [13]:
def make_body(table_name):
    pks = primary_key[table_name]
    fks = foreing_key[table_name]
    columns = columns_all[table_name]
    # sort columns
    columns = dict(sorted(columns.items(), key=lambda item: (item[1], item[0])))

    mapped = []
    body_list = []
    if len(pks) > 0:
        key = pks[0]
        body_PK = primary_key_example.replace('PRIMARY_KEY', key)
        body_PK = body_PK.replace('COLUMN_TYPE', columns[key])
        body_list.append(body_PK)
        mapped.append(key)
    if len(fks) > 0:
        for key in fks:
            if key not in mapped:
                body_FK = colum_example.replace('COLUMN_NAME', key)
                body_FK = body_FK.replace('COLUMN_TYPE', columns[key])
                body_list.append(body_FK)
                mapped.append(key)
    for key in columns:
        if key not in mapped:
            body_column = colum_example.replace('COLUMN_NAME', key)
            body_column = body_column.replace('COLUMN_TYPE', columns[key])
            body_list.append(body_column)
            mapped.append(key)
    body_all = '\n'.join(body_list)
    return body_all

In [14]:
def make_table(table_name):
    print(f'{table_name}')
    body_all = make_body(table_name)
    table_code = table_code_example.replace('TABLE_NAME', table_name)
    table_code = table_code.replace('TABLE_BODY', body_all)
    return table_code

In [15]:
script_table_list = []
for table_name in columns_all.keys():
    table_code = make_table(table_name)
    script_table_list.append(table_code)
len(script_table_list)

00_df_general
01_df_tomb
02_df_tomb_king_name
03_df_tomb_chapel
04_df_tomb_osiris
05_df_tomb_statue_material
06_df_tomb_dim
07_df_name
08_df_person
09_df_person_working_activity
10_df_shaft_burial_chamber
11_df_shaft_sarcophagus
12_df_shaft_coffin
13_df_tomb_shaft_relation
14_df_shaft_burial_chamber_dim
15_df_body
16_df_body_housed
17_df_source_type
18_df_notes
19_df_current_location
20_df_dating
21_df_reference
22_df_bib_reference
23_df_kings_info
24_df_coffin_decoration
25_df_general_coffin_dec
26_df_sarcophagus_decoration
27_df_general_sarcophagus_dec
28_df_attributes
29_df_general_attributes
30_df_attributes_position
31_df_equipment
32_df_general_equipment
33_df_equipment_material
34_df_family_relations
35_df_titles
36_df_titles_other_reference
37_df_titles_general
38_df_person_title
39_df_false_door
40_df_false_door_family_members
41_df_false_door_titles_highlighted
42_df_false_door_festivals
43_df_false_door_general


44

In [16]:
body_all = '\n\n'.join(script_table_list)
body_all = body_all.replace('-', '')

# save body as txt file
import os
output_file = 'database_schema_print.txt'
path = os.path.join('output', output_file)
with open(path, "w") as file:
    file.write(body_all)

print(body_all)

Table 00_df_general {
	ID_official BIGINT [primary key]
	ID_person BIGINT
	king TEXT
	name_id BIGINT
	tomb_code_id TEXT
	buried TEXT
	dynasty TEXT
	find_number TEXT
	locality TEXT
	mentioned TEXT
	nome TEXT
	owner TEXT
	site TEXT
	uncertain TEXT
}

Table 01_df_tomb {
	tomb_code_id TEXT [primary key]
	catalogized BOOLEAN
	statues FLOAT
	A40_great_god TEXT
	ch_false_door TEXT
	ch_workmanship TEXT
	curiosity TEXT
	embracing TEXT
	excavation_year TEXT
	excavator TEXT
	label_of_shaft TEXT
	number_of_shafts TEXT
	presence_of_wife TEXT
	provenance TEXT
	servant_statues TEXT
	spacial_distribution TEXT
	tomb_code TEXT
	tomb_type TEXT
	type_of_decoration TEXT
}

Table 02_df_tomb_king_name {
	ID_official BIGINT
	tomb_code_id TEXT
	dynasty TEXT
	source TEXT
}

Table 03_df_tomb_chapel {
	tomb_code_id TEXT
	chapel_area FLOAT
	chapel_dimension_certainty TEXT
	chapel_dimensions TEXT
	length TEXT
	width TEXT
}

Table 04_df_tomb_osiris {
	tomb_code_id TEXT
	Osiris TEXT
}

Table 05_df_tomb_statue_materia

### make ref script for dbdiagram.io

In [17]:
ref_example = 'Ref: TABLE_NAME_1.COLUMN_1 SIGN TABLE_NAME_2.COLUMN_2'

In [18]:
foreing_key

{'00_df_general': ['ID_official',
  'ID_person',
  'king',
  'name_id',
  'tomb_code_id'],
 '01_df_tomb': ['tomb_code_id'],
 '02_df_tomb_king_name': ['ID_official', 'tomb_code_id'],
 '03_df_tomb_chapel': ['tomb_code_id'],
 '04_df_tomb_osiris': ['tomb_code_id'],
 '05_df_tomb_statue_material': ['tomb_code_id'],
 '06_df_tomb_dim': ['tomb_code_id'],
 '07_df_name': ['ID_official_source', 'name_id'],
 '08_df_person': ['ID_person'],
 '09_df_person_working_activity': ['ID_person'],
 '10_df_shaft_burial_chamber': ['ID_official',
  'coffin_ID',
  'sarcophagus_ID',
  'shaft_ID'],
 '11_df_shaft_sarcophagus': ['sarcophagus_ID'],
 '12_df_shaft_coffin': ['coffin_ID'],
 '13_df_tomb_shaft_relation': ['ID_person', 'shaft_ID', 'tomb_code_id'],
 '14_df_shaft_burial_chamber_dim': ['shaft_ID'],
 '15_df_body': ['shaft_ID'],
 '16_df_body_housed': ['shaft_ID'],
 '17_df_source_type': ['ID_official'],
 '18_df_notes': ['ID_official'],
 '19_df_current_location': ['ID_official'],
 '20_df_dating': ['ID_official'],
 

In [19]:
primary_key = primary_keys.copy()
foreing_key = {table_name: sorted([id
                            for id in ids if id not in primary_key[table_name]])
               for table_name, ids in table_id_dict.items()}
pk_to_table = {pk[0]:table_name for table_name, pk in primary_key.items() if len(pk)>0}

In [20]:
ref_exceptions = {
    'ID_official_source': 'ID_official',
    'ID_official_1': 'ID_official', 
    'ID_official_2': 'ID_official',
    'ID_person_1': 'ID_person',
    'ID_person_2': 'ID_person'
}

In [21]:
error_list = []
ref_all = []
for table_name, ids in foreing_key.items():
    if len(ids) > 0:
        for id in ids:
            # if foreing key is mapped as primary key in any table
            if id  in list(pk_to_table.keys()):
                primary_id = id
            # if no table with correposing primary key was found
            # -> use expetions
            else:
                primary_id = ref_exceptions.get(id, None)
            # if even after exception primary key was not found
            # -> save error
            if primary_id is None:
                error = [table_name, id]
                error_list.appned(error)
                print(f'ERROR:\t{error}')
                continue
            
            primary_table_name = pk_to_table[primary_id]
            ref_code = ref_example.replace('TABLE_NAME_1', table_name)
            ref_code = ref_code.replace('COLUMN_1', id)
            ref_code = ref_code.replace('TABLE_NAME_2', primary_table_name)
            ref_code = ref_code.replace('COLUMN_2', primary_id)
            ref_all.append(ref_code)
    #print('\n', table_name)
print(error_list)

[]


In [22]:
print(f'Sum of foreing keys properties: {sum([len(ids) for ids in foreing_key.values()])}')
print(f'Sum of all refernces: {len(ref_all)}')

Sum of foreing keys properties: 52
Sum of all refernces: 52


In [23]:
# sord ref
# serach for first two digits of format XY
pattern = re.compile(r"(\d{2})_.*?(\d{2})_")
ref_all = sorted(ref_all, key=lambda x: tuple(map(int, pattern.search(x).groups())))

ref_body = '\n\n'.join(ref_all)
ref_body = ref_body.replace('SIGN', '<>')

# save body as txt file
import os
output_file = 'database_ref_print.txt'
path = os.path.join('output', output_file)
with open(path, "w") as file:
    file.write(ref_body)

print(ref_body)

Ref: 00_df_general.tomb_code_id <> 01_df_tomb.tomb_code_id

Ref: 00_df_general.name_id <> 07_df_name.name_id

Ref: 00_df_general.ID_person <> 08_df_person.ID_person

Ref: 00_df_general.king <> 23_df_kings_info.king

Ref: 02_df_tomb_king_name.ID_official <> 00_df_general.ID_official

Ref: 02_df_tomb_king_name.tomb_code_id <> 01_df_tomb.tomb_code_id

Ref: 03_df_tomb_chapel.tomb_code_id <> 01_df_tomb.tomb_code_id

Ref: 04_df_tomb_osiris.tomb_code_id <> 01_df_tomb.tomb_code_id

Ref: 05_df_tomb_statue_material.tomb_code_id <> 01_df_tomb.tomb_code_id

Ref: 06_df_tomb_dim.tomb_code_id <> 01_df_tomb.tomb_code_id

Ref: 07_df_name.ID_official_source <> 00_df_general.ID_official

Ref: 09_df_person_working_activity.ID_person <> 08_df_person.ID_person

Ref: 10_df_shaft_burial_chamber.ID_official <> 00_df_general.ID_official

Ref: 10_df_shaft_burial_chamber.sarcophagus_ID <> 11_df_shaft_sarcophagus.sarcophagus_ID

Ref: 10_df_shaft_burial_chamber.coffin_ID <> 12_df_shaft_coffin.coffin_ID

Ref: 13_df_